In [1]:
import sys
from unicodedata import name
from mozaik.controller import Global, setup_logging
from mozaik.storage.datastore import PickledDataStore
from parameters import ParameterSet
from mozaik.storage.queries import param_filter_query
import mozaik
import os
import gc
import pickle
import numpy as np
from mozaik.tools.mozaik_parametrized import MozaikParametrized
from tqdm import tqdm
import logging
import imagen 
from imagen.image import BoundingBox
from mozaik.stimuli.vision.topographica_based import MaximumDynamicRange

In [2]:
logging.basicConfig(stream=sys.stdout, level=logging.ERROR)


In [3]:
def get_datastore(root):
    Global.root_directory = root
    datastore = PickledDataStore(
        load=True,
        parameters=ParameterSet({"root_directory": root, "store_stimuli": False}),
        replace=True,
    )
    return datastore


In [4]:
def extract_images(path):
    path = path.split("from_")[1].split("_")[0]
    return path

In [5]:
def pickledump(path, file):
    with open(path, "wb") as f:
        pickle.dump(file, f)


In [6]:
def reconstruct_stimuli(s):
    pattern_sampler = imagen.image.PatternSampler(
            size_normalization="fit_shortest",
            whole_pattern_output_fns=[MaximumDynamicRange()],
        )

    img = imagen.image.FileImage(
        filename=s.image_path,
        x=0,
        y=0,
        orientation=0,
        xdensity=s.density,
        ydensity=s.density,
        size=s.size,
        bounds=BoundingBox(
            points=(
                (-s.size_x / 2, -s.size_y / 2),
                (s.size_x / 2, s.size_y / 2),
            )
        ),
        scale=2 * s.background_luminance,
        pattern_sampler=pattern_sampler,
    )
    return img()

In [7]:
def get_sheetname(sheet):
    if sheet == "V1_Inh_L2/3":
        sheet = "V1_Inh_L23"
    if sheet == "V1_Exc_L2/3":
        sheet= "V1_Exc_L23"
    return sheet

In [8]:
# run export

# path = sys.argv[1]
# sheet = sys.argv[2]
sheet = 'V1_Exc_L2/3'
path = '/CSNG/baroni/mozaik-models/LSV1M/20240116-093251[param_nat_img.defaults]CombinationParamSearch{trial:[0],baseline:[0]}/NewDataset_Images_from_0_to_100_ParameterSearch_____baseline:0_trial:0'
datastore = get_datastore(path)

dsv = param_filter_query(datastore, st_name='NaturalImage')
sheets = ['V1_Exc_L2/3', 'V1_Inh_L2/3', 'V1_Exc_L4', 'V1_Inh_L4', 'X_ON' 'X_OFF']
sheet_folders  = ['V1_Exc_L23', 'V1_Inh_L23', 'V1_Exc_L4', 'V1_Inh_L4', 'X_ON' 'X_OFF']

trials = sorted(list(set( MozaikParametrized.idd(s).trial for s in dsv.get_stimuli())))
img_paths =  sorted(list(set(MozaikParametrized.idd(s).image_path for s in dsv.get_stimuli())))

setup_logging()
logger = mozaik.getMozaikLogger()

In [9]:
datastore
print(len(trials))

1


In [18]:
def get_segments(dsv):
    """
    Retrieves segments for both blanks and images in chronological order as they 
    were presented in the experiments.
    :param dsv: datastore containing data information.
    :returns: Tuple of segments for blanks and images from `dsv`.
    """
    segs_blank = dsv.get_segments(null=True,ordered=True)
    segs_image = dsv.get_segments(ordered=True)

    return segs_blank, segs_image

In [19]:
def get_image_number(segment):
    """
    Retrieves number of the image of the given segment.
    :param segment: segment to obtain image information from.
    :returns: Number of the image corresponding to segment.
    """
    stimulus = MozaikParametrized.idd(segment.annotations['stimulus'])
    return stimulus.image_path.split('/')[-1].split('_')[0]

In [ ]:
def get_spike_times(

In [29]:
def sort_spiketrains(spike_trains):
    
    def sorting_key(spike_train):
        # Calculate the average firing rate for each SpikeTrain
        return spike_train.annotations['source_id']
    
    # Sort the SpikeTrain objects based on the average firing rate
    return sorted(spike_trains, key=sorting_key)

In [33]:

if len(trials) == 1:
    print(f'There is a single trial')
    dsv1 = param_filter_query(dsv, sheet_name = sheet)
    for trial in trials:
        dsv2 = param_filter_query(dsv1, st_trial = trial)
        # segs_stimuli = dsv2.get_segments(ordered=True)
        # segs_blank = dsv2.get_segments(null=True,ordered=True)

        # Should be chronologically sorted segments.
        segs_blank, segs_images = get_segments(dsv2)

        segs_blank = segs_blank[0:2]
        segs_stimuli = segs_images[0:2]
        for seg_blank, seg_image in zip(segs_blank, segs_stimuli):
            # stim_blank = MozaikParametrized.idd(seg_blank.annotations['stimulus'])  
            # stim_stimuli = MozaikParametrized.idd(seg_stimuli.annotations['stimulus'])

            # img_number_blank = stim_blank.image_path.split('/')[-1].split('_')[0]
            # img_number_stimuli = stim_stimuli.image_path.split('/')[-1].split('_')[0]

            # segs_stimuli[0].spiketrains[2].annotations['source_index']

            image_number = get_image_number(seg_blank)
            print("NEW TRIAL")
            print("-------------------------------------------")
            # print(stim_blank)
            # print(stim_stimuli)
            # print()
            # print(img_number_blank)
            print(f"Image number: {img_number_stimuli}")
            print()

            print(type(seg_blank.spiketrains))

            sorted_blank_spiketrains = sort_spiketrains(seg_blank.spiketrains[0:2])
            sorted_images_spiketrains = sort_spiketrains(seg_image.spiketrains[0:2])

            blank_spiketrains = [np.array(st) for st in sorted_blank_spiketrains]
            image_spiketrains = [np.array(st) for st in sorted_images_spiketrains]
            print(blank_spiketrains)
            print(image_spiketrains)
            
            for spikes_blank, spikes_stimuli in zip(
                    sorted_blank_spiketrains,
                    sorted_images_spiketrains
                    #seg_blank.spiketrains[0:2],
                    #seg_stimuli.spiketrains[0:2]
                ):
                print(f"Neuron ID: {spikes_blank.annotations['source_id']}")
                print(type(spikes_blank))
                print(f"Blank spikes: {spikes_blank}")
                print(f"Stimuli spikes: {spikes_stimuli}")
                # print(f"Blank neuron: {spikes_stimuli.annotations['source_id']}")
                print()

        
        
            # for seg in tqdm(segs):
            #     stim = MozaikParametrized.idd(seg.annotations['stimulus'])
                
            #     img_number = stim.image_path.split('/')[-1].split('_')[0]
            #     resp_path = os.path.join('/home/beinhaud', 'single_trial', img_number)
            #     print("NEW TRIAL")
            #     print(seg)
            #     resp = seg.mean_rates()
            #     # print(resp.shape)
    
            #     gc.collect()
                
            #     os.makedirs(resp_path, exist_ok=True)
            #     np.save(os.path.join(resp_path, get_sheetname(sheet) +'.npy'), resp)
            #     if trial == 0:
            #         if sheet == sheets[0]:
            #             img = reconstruct_stimuli(stim)
            #             np.save(os.path.join(resp_path, 'stimulus' +'.npy'), img)
            #             gc.collect()

        

            # stims = dsv2.get_stimuli()
            # stims_n = [MozaikParametrized.idd(st).image_path.split('/')[-1].split('_')[0] for st in stims]

            # # needs to be fixed
            # segs = [seg for _, seg in sorted(zip(stims_n, segs))]
            
            # gc.collect()
            # resps = [s.mean_rates() for s in tqdm(segs)]
            # gc.collect()
            # for i, stim in enumerate(tqdm(stims)):
            #     parametrized_stim = MozaikParametrized.idd(stim)
            #     img_number = parametrized_stim.image_path.split('/')[-1].split('_')[0]
            #     resp_path = os.path.join('/CSNG/baroni/test/', 'single_trial', img_number)
            #     os.makedirs(resp_path, exist_ok=True)
            #     np.save(os.path.join(resp_path, sf +'.npy'), resps[i])
            #     if trial ==0:
            #         if sheet == sheets[0]:
            #             img = reconstruct_stimuli(parametrized_stim)
            #             np.save(os.path.join(resp_path, 'stimulus' +'.npy'), img)
            #             gc.collect()

There is a single trial
NEW TRIAL
-------------------------------------------
Image number: 0000000090

<class 'list'>
[array([], dtype=float64), array([35.8])]
[array([], dtype=float64), array([515.6])]
Neuron ID: 61286
<class 'neo.core.spiketrain.SpikeTrain'>
Blank spikes: [] ms
Stimuli spikes: [] ms

Neuron ID: 61287
<class 'neo.core.spiketrain.SpikeTrain'>
Blank spikes: [35.8] ms
Stimuli spikes: [515.6] ms

NEW TRIAL
-------------------------------------------
Image number: 0000000090

<class 'list'>
[array([], dtype=float64), array([], dtype=float64)]
[array([175.8, 180.2, 216.8, 318.6]), array([], dtype=float64)]
Neuron ID: 61286
<class 'neo.core.spiketrain.SpikeTrain'>
Blank spikes: [] ms
Stimuli spikes: [175.8 180.2 216.8 318.6] ms

Neuron ID: 61287
<class 'neo.core.spiketrain.SpikeTrain'>
Blank spikes: [] ms
Stimuli spikes: [] ms



In [27]:
spikes_blank.


SpikeTrain containing 0 spikes; units ms; datatype float64 
annotations: {'source_population': 'V1_Exc_L2/3',
  'source_id': 61287,
  'source_index': 1,
  'channel_id': 61287}
time: 0.0 ms to 150.0 ms

In [11]:
dsv2.get_stimuli(ordered=True)[0:2]

['{"module_path" :"mozaik.stimuli.vision.topographica_based","background_luminance":50.0, "blank_duration":0.0, "density":20.0, "direct_stimulation_name":None, "direct_stimulation_parameters":None, "duration":560, "frame_duration":7.0, "image_duration":560.0, "image_path":\'/projects/ImageDatasets/imagenet/all_imagenet_images/0000000069_n02093754_1901.JPEG\', "location_x":0.0, "location_y":0.0, "name":\'NaturalImage\', "size":11, "size_x":11.0, "size_y":11.0, "trial":0}',
 '{"module_path" :"mozaik.stimuli.vision.topographica_based","background_luminance":50.0, "blank_duration":0.0, "density":20.0, "direct_stimulation_name":None, "direct_stimulation_parameters":None, "duration":560, "frame_duration":7.0, "image_duration":560.0, "image_path":\'/projects/ImageDatasets/imagenet/all_imagenet_images/0000000090_n02007558_12982.JPEG\', "location_x":0.0, "location_y":0.0, "name":\'NaturalImage\', "size":11, "size_x":11.0, "size_y":11.0, "trial":0}']

In [12]:
print(len(segs_stimuli))
print(len(segs_blank))
# print(segs_stimuli[0])
# print(segs_blank[0])
# segs_stimuli[0]

2
2


In [13]:
segs_stimuli[0].spiketrains[0:5]

[SpikeTrain containing 0 spikes; units ms; datatype float64 
 annotations: {'source_population': 'V1_Exc_L2/3',
   'source_id': 61286,
   'source_index': 0,
   'channel_id': 61286}
 time: 0.0 ms to 560.0 ms,
 SpikeTrain containing 1 spikes; units ms; datatype float64 
 annotations: {'source_population': 'V1_Exc_L2/3',
   'source_id': 61287,
   'source_index': 1,
   'channel_id': 61287}
 time: 0.0 ms to 560.0 ms,
 SpikeTrain containing 2 spikes; units ms; datatype float64 
 annotations: {'source_population': 'V1_Exc_L2/3',
   'source_id': 61288,
   'source_index': 2,
   'channel_id': 61288}
 time: 0.0 ms to 560.0 ms,
 SpikeTrain containing 0 spikes; units ms; datatype float64 
 annotations: {'source_population': 'V1_Exc_L2/3',
   'source_id': 61289,
   'source_index': 3,
   'channel_id': 61289}
 time: 0.0 ms to 560.0 ms,
 SpikeTrain containing 0 spikes; units ms; datatype float64 
 annotations: {'source_population': 'V1_Exc_L2/3',
   'source_id': 61290,
   'source_index': 4,
   'channel

In [14]:
segs_blank[0].spiketrains[0:5]

[SpikeTrain containing 0 spikes; units ms; datatype float64 
 annotations: {'source_population': 'V1_Exc_L2/3',
   'source_id': 61286,
   'source_index': 0,
   'channel_id': 61286}
 time: 0.0 ms to 150.0 ms,
 SpikeTrain containing 1 spikes; units ms; datatype float64 
 annotations: {'source_population': 'V1_Exc_L2/3',
   'source_id': 61287,
   'source_index': 1,
   'channel_id': 61287}
 time: 0.0 ms to 150.0 ms,
 SpikeTrain containing 0 spikes; units ms; datatype float64 
 annotations: {'source_population': 'V1_Exc_L2/3',
   'source_id': 61288,
   'source_index': 2,
   'channel_id': 61288}
 time: 0.0 ms to 150.0 ms,
 SpikeTrain containing 0 spikes; units ms; datatype float64 
 annotations: {'source_population': 'V1_Exc_L2/3',
   'source_id': 61289,
   'source_index': 3,
   'channel_id': 61289}
 time: 0.0 ms to 150.0 ms,
 SpikeTrain containing 1 spikes; units ms; datatype float64 
 annotations: {'source_population': 'V1_Exc_L2/3',
   'source_id': 61290,
   'source_index': 4,
   'channel

In [15]:
type(segs_stimuli[0].spiketrains)
segs_stimuli[0].spiketrains[2].times

array([272., 464.]) * ms

In [16]:
segs_stimuli[0].spiketrains[2].annotations['source_index']

2

In [17]:
segs_blank[0].spiketrains[2].times

array([], dtype=float64) * ms

In [ ]:
# if len(trials) != 1:
#     print(f'There are multiple trials ({len(trials)})')
#     for sheet, sf in zip(sheets, sheet_folders):
#         print(f'sheet={sheet}')
#         dsv1 = param_filter_query(dsv, sheet_name = sheet)
#         for trial in trials:
#             print(f'trial={trial}')
#             dsv2 = param_filter_query(dsv1, st_trial = trial)
#             segs = dsv2.get_segments()
#             stims = dsv2.get_stimuli()
#             stims_n = [MozaikParametrized.idd(st).image_path.split('/')[-1].split('_')[0] for st in stims]
#             segs = [seg for _, seg in sorted(zip(stims_n, segs))]

#             gc.collect()
#             resps = [s.mean_rates() for s in tqdm(segs)]
#             gc.collect()
#             for i, stim in enumerate(tqdm(stims)):
#                 parametrized_stim = MozaikParametrized.idd(stim)
#                 img_number = parametrized_stim.image_path.split('/')[-1].split('_')[0]
#                 resp_path = os.path.join('/home/baroni/mozaik-models_v1paper/exported_data', 'multitrial', img_number)
#                 resp_path_trial = os.path.join('/home/baroni/mozaik-models_v1paper/exported_data', 'multitrial', img_number, 'trial='+str(trial))
#                 os.makedirs(resp_path_trial, exist_ok=True)
#                 np.save(os.path.join(resp_path_trial, sf +'.npy'), resps[i])
#                 if trial ==0:
#                     if sheet == sheets[0]:
#                         img = reconstruct_stimuli(parametrized_stim)
#                         np.save(os.path.join(resp_path, 'stimulus' +'.npy'), img)
#                         gc.collect()